# Dataset Validation for External/Custom Datasets

This notebook demonstrates how to validate external or custom datasets using `ydata-profiling`.

Contents:
- Requirements and installation
- Loading data from various sources (CSV, Excel, JSON, Parquet)
- Data preprocessing and transformation
- Generating comprehensive data profile reports
- Customising the validation report
- Comparing datasets
- Exporting reports for documentation

**Note:** This notebook is designed for datasets that come from external sources and may require preprocessing before validation.

## Requirements

- Python 3.9+
- `ydata-profiling` package (dataset profiling)
- `pandas` package (data manipulation)
- `ipywidgets` package (for interactive widgets)
- Optional: `openpyxl` (Excel files), `pyarrow` (Parquet files)

Quick installation:
```bash
pip install ydata-profiling pandas openpyxl pyarrow ipywidgets
```

In [ ]:
!pip install ydata-profiling pandas openpyxl pyarrow ipywidgets

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import os
from datetime import datetime

print(f"pandas version: {pd.__version__}")

## Loading Data from Various Sources

Choose the appropriate method based on your data source. Uncomment and modify the relevant section.

### Option 1: Load from CSV file

In [ ]:
# Load from CSV file
# Adjust the path and parameters to match your file

CSV_PATH = "./your_dataset.csv"

# Common CSV parameters you may need to adjust:
df = pd.read_csv(
    CSV_PATH,
    # sep=";",           # Delimiter (default is comma)
    # encoding="utf-8",  # File encoding
    # header=0,          # Row number to use as column names
    # skiprows=1,        # Rows to skip at the beginning
    # na_values=["NA", "N/A", ""],  # Additional strings to recognise as NA
    # parse_dates=["date_column"],  # Columns to parse as dates
    # dtype={"column_name": str},   # Specify column data types
)

print(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

### Option 2: Load from Excel file

In [ ]:
# Load from Excel file
# Uncomment and adjust the path and parameters

# EXCEL_PATH = "./your_dataset.xlsx"
# 
# df = pd.read_excel(
#     EXCEL_PATH,
#     sheet_name=0,      # Sheet name or index (0 for first sheet)
#     # header=0,        # Row number to use as column names
#     # skiprows=1,      # Rows to skip at the beginning
#     # usecols="A:F",   # Columns to read (Excel-style range)
# )
# 
# print(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

### Option 3: Load from JSON file

In [ ]:
# Load from JSON file
# Uncomment and adjust the path and parameters

# JSON_PATH = "./your_dataset.json"
# 
# # For standard JSON array of objects:
# df = pd.read_json(JSON_PATH)
# 
# # For JSON lines format (one JSON object per line):
# # df = pd.read_json(JSON_PATH, lines=True)
# 
# # For nested JSON, you may need to normalise:
# # import json
# # with open(JSON_PATH, 'r') as f:
# #     data = json.load(f)
# # df = pd.json_normalize(data, record_path=['nested_key'])
# 
# print(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

### Option 4: Load from Parquet file

In [ ]:
# Load from Parquet file
# Uncomment and adjust the path

# PARQUET_PATH = "./your_dataset.parquet"
# 
# df = pd.read_parquet(PARQUET_PATH)
# 
# print(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

### Option 5: Load from URL

In [ ]:
# Load from URL (CSV example)
# Uncomment and adjust the URL

# DATA_URL = "https://example.com/your_dataset.csv"
# 
# df = pd.read_csv(DATA_URL)
# 
# print(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

### Example: Using a sample dataset for demonstration

In [ ]:
# Example: Create a sample dataset for demonstration
# Remove this cell when using your own data

from sklearn.datasets import fetch_california_housing

print("Loading sample dataset for demonstration...")
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['target'] = housing.target

# Add some synthetic issues for demonstration
np.random.seed(42)
df.loc[np.random.choice(df.index, 100), 'AveRooms'] = np.nan  # Add missing values
df.loc[np.random.choice(df.index, 50), 'AveBedrms'] = np.nan
df = pd.concat([df, df.iloc[:20]])  # Add duplicates

print(f"Sample data loaded: {len(df)} rows, {len(df.columns)} columns")

## Data Preview and Initial Inspection

In [ ]:
# Preview the data
print("Data Preview (first 10 rows):")
df.head(10)

In [ ]:
# DataFrame info
print("DataFrame Info:")
df.info()

print("\nColumn Data Types:")
print(df.dtypes)

In [ ]:
# Basic statistics
print("Basic Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values Summary:")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

## Data Preprocessing (Optional)

Apply transformations to prepare your data for profiling. These steps are optional and depend on your data quality requirements.

### Handling Data Types

In [ ]:
# Convert columns to appropriate data types
# Uncomment and adjust as needed

# Convert to datetime
# df['date_column'] = pd.to_datetime(df['date_column'], format='%Y-%m-%d')

# Convert to numeric
# df['numeric_column'] = pd.to_numeric(df['numeric_column'], errors='coerce')

# Convert to categorical
# df['category_column'] = df['category_column'].astype('category')

# Convert to string
# df['string_column'] = df['string_column'].astype(str)

print("Data types after conversion:")
print(df.dtypes)

### Handling Missing Values

In [ ]:
# Options for handling missing values (uncomment as needed)

# Option 1: Drop rows with any missing values
# df_clean = df.dropna()

# Option 2: Drop rows with missing values in specific columns
# df_clean = df.dropna(subset=['important_column1', 'important_column2'])

# Option 3: Fill missing values with a specific value
# df['column_name'] = df['column_name'].fillna(0)

# Option 4: Fill missing values with mean/median/mode
# df['numeric_column'] = df['numeric_column'].fillna(df['numeric_column'].mean())

# Option 5: Forward/backward fill
# df['column_name'] = df['column_name'].fillna(method='ffill')

print("Missing values handling: No changes applied (modify as needed)")

### Handling Duplicates

In [ ]:
# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Uncomment to remove duplicates
# df = df.drop_duplicates()
# print(f"After removing duplicates: {len(df)} rows")

### Column Renaming and Selection

In [ ]:
# Rename columns (uncomment and adjust as needed)
# df = df.rename(columns={
#     'old_name1': 'new_name1',
#     'old_name2': 'new_name2',
# })

# Select specific columns
# df = df[['column1', 'column2', 'column3']]

# Drop specific columns
# df = df.drop(columns=['unnecessary_column1', 'unnecessary_column2'])

print(f"Current columns: {list(df.columns)}")

## Generate Data Profile Report

Use `ydata-profiling` to generate a comprehensive data quality report.

In [ ]:
# Generate profile report
print("Generating data profile report...")
print("This may take a few minutes depending on dataset size.")

profile = ProfileReport(
    df,
    title="External Dataset Validation Report",
    explorative=True,
)

print("Profile report generated successfully.")

In [ ]:
# Display the report in the notebook
# Using to_notebook_iframe() for better compatibility
profile.to_notebook_iframe()

## Customising the Report

### Minimal Mode (for large datasets)

In [ ]:
# Minimal profile for large datasets
profile_minimal = ProfileReport(
    df,
    title="External Dataset - Minimal Report",
    minimal=True,  # Faster, less detailed
)

profile_minimal.to_notebook_iframe()

### Custom Configuration

In [ ]:
# Custom profile configuration
profile_custom = ProfileReport(
    df,
    title="External Dataset - Custom Report",
    # Correlation settings
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
    },
    # Missing values visualization
    missing_diagrams={
        "bar": True,
        "matrix": True,
        "heatmap": True,
    },
    # Sample size for interactions
    interactions={"continuous": False},
    # Duplicates detection
    duplicates={"head": 10},
)

profile_custom.to_notebook_iframe()

### Time Series Mode

If your dataset contains time series data, use the time series mode for specialised analysis.

In [ ]:
# Time series profile (uncomment if your data is time series)
# Ensure your DataFrame has a datetime index or column

# # Set datetime column as index
# df_ts = df.set_index('datetime_column')
# 
# profile_ts = ProfileReport(
#     df_ts,
#     title="External Dataset - Time Series Report",
#     tsmode=True,
#     sortby="datetime_column",  # Column to sort by
# )
# 
# profile_ts.to_notebook_iframe()

print("Time series mode: Uncomment the cell above if your data is time series.")

## Comparing Two Datasets

Compare two versions of a dataset (e.g., before/after cleaning, train/test split).

In [ ]:
# Example: Compare original vs cleaned dataset
# Uncomment and adjust as needed

# # Create a "cleaned" version for comparison
# df_cleaned = df.dropna().drop_duplicates()
# 
# # Generate profiles for both datasets
# profile_original = ProfileReport(df, title="Original Dataset")
# profile_cleaned = ProfileReport(df_cleaned, title="Cleaned Dataset")
# 
# # Generate comparison report
# comparison_report = profile_original.compare(profile_cleaned)
# comparison_report.to_notebook_iframe()

print("Dataset comparison: Uncomment the cell above to compare datasets.")

## Export Reports

In [ ]:
# Create reports directory
REPORTS_DIR = "./reports"
os.makedirs(REPORTS_DIR, exist_ok=True)

# Generate timestamp for file names
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Export to HTML (self-contained, shareable)
html_path = f"{REPORTS_DIR}/external_dataset_validation_{timestamp}.html"
profile.to_file(html_path)
print(f"HTML report saved to: {html_path}")

# Export to JSON (for programmatic analysis)
json_path = f"{REPORTS_DIR}/external_dataset_validation_{timestamp}.json"
profile.to_file(json_path)
print(f"JSON report saved to: {json_path}")

## Accessing Report Data Programmatically

In [ ]:
# Get the profile description as a dictionary
description = profile.get_description()

# Dataset overview
print("Dataset Overview:")
print(f"  Number of variables: {description.table['n_var']}")
print(f"  Number of observations: {description.table['n']}")
print(f"  Missing cells: {description.table['n_cells_missing']}")
print(f"  Missing cells (%): {description.table['p_cells_missing']:.2%}")
print(f"  Duplicate rows: {description.table['n_duplicates']}")
print(f"  Duplicate rows (%): {description.table['p_duplicates']:.2%}")

In [ ]:
# Variable-level statistics
print("\nVariable Types:")
for var_type, count in description.table['types'].items():
    print(f"  {var_type}: {count}")

# Alerts/warnings
print("\nAlerts:")
alerts = description.alerts
if alerts:
    for alert in alerts:
        print(f"  [{alert.alert_type.name}] {alert.column_name}: {alert.alert_type_name}")
else:
    print("  No alerts found.")

## Data Quality Checklist

Use this checklist to validate your dataset before using it for model training:

- [ ] **Completeness**: Missing values are within acceptable limits
- [ ] **Uniqueness**: Duplicate records have been addressed
- [ ] **Validity**: Data types are correct for each column
- [ ] **Accuracy**: Values are within expected ranges
- [ ] **Consistency**: Related columns have consistent values
- [ ] **Timeliness**: Data is current and relevant

In [ ]:
# Automated data quality checks
print("=" * 60)
print("DATA QUALITY SUMMARY")
print("=" * 60)

# Completeness
missing_pct = description.table['p_cells_missing'] * 100
completeness_status = "PASS" if missing_pct < 5 else "WARN" if missing_pct < 20 else "FAIL"
print(f"\nCompleteness: {completeness_status}")
print(f"  Missing cells: {missing_pct:.2f}%")

# Uniqueness
duplicate_pct = description.table['p_duplicates'] * 100
uniqueness_status = "PASS" if duplicate_pct < 1 else "WARN" if duplicate_pct < 5 else "FAIL"
print(f"\nUniqueness: {uniqueness_status}")
print(f"  Duplicate rows: {duplicate_pct:.2f}%")

# Alerts summary
alert_count = len(description.alerts)
alerts_status = "PASS" if alert_count == 0 else "WARN" if alert_count < 5 else "FAIL"
print(f"\nAlerts: {alerts_status}")
print(f"  Total alerts: {alert_count}")

print("\n" + "=" * 60)

## Tips and Troubleshooting

- **Large datasets**: Use `minimal=True` or sample your data:
  ```python
  df_sample = df.sample(n=10000, random_state=42)
  profile = ProfileReport(df_sample, minimal=True)
  ```

- **Memory issues**: Reduce dataset size or disable heavy computations:
  ```python
  profile = ProfileReport(df, interactions={"continuous": False}, correlations=None)
  ```

- **Encoding issues with CSV**: Try different encodings:
  ```python
  df = pd.read_csv(path, encoding='latin-1')  # or 'cp1252', 'iso-8859-1'
  ```

- **Date parsing issues**: Specify the format explicitly:
  ```python
  df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
  ```

- **High cardinality categorical columns**: Limit unique values shown:
  ```python
  profile = ProfileReport(df, vars={"cat": {"n_obs": 5}})
  ```

- **Widget rendering issues**: If `to_widgets()` fails with compatibility errors, use `to_notebook_iframe()` instead for reliable HTML rendering in the notebook.